<a href="https://colab.research.google.com/github/Prasadkurapati7/Data-Mining-/blob/main/Data_Mining_A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
# Shubham Patel - Student ID : 916193675

In [62]:
!pip install timm

In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
import pandas as pd
import cv2
import os
import numpy as np
import warnings
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, SpectralClustering, BisectingKMeans
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from sklearn.metrics import fowlkes_mallows_score, silhouette_score


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


import timm

import albumentations as A
from albumentations.pytorch import ToTensorV2

In [65]:
warnings.filterwarnings('ignore')

# 1. Feature Extraction

In [66]:
path = "/content/drive/MyDrive/DM/updated"

In [67]:
class_names = ['Malamute','Kerry_blue_terrier','German_short-haired_pointer','Welsh_springer_spaniel']

actual_classes = ['n02110063-malamute','n02093859-Kerry_blue_terrier','n02100236-German_short-haired_pointer','n02102177-Welsh_springer_spaniel']

In [68]:
df = pd.DataFrame(columns = ['image_id','label'])
for class_ in actual_classes:
  class_path = os.path.join(path,class_)
  for filename in os.listdir(class_path):
    df.loc[len(df)] = [filename,actual_classes.index(class_)]


In [69]:
df.head()

,image_id,label
0,n02110063_13152.jpg,0
1,n02110063_6276.jpg,0
2,n02110063_17073.jpg,0
3,n02110063_609.jpg,0
4,n02110063_15580.jpg,0


In [70]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [71]:
class ImageData(Dataset):


    def __init__(self, data, directory, transform,actual_classes):
        self.data = data
        self.directory = directory
        self.transform = transform
        self.actual_classes = actual_classes


    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):


        path = os.path.join(self.directory,actual_classes[self.data.iloc[idx]['label']])
        image = cv2.imread(
            os.path.join(path, self.data.iloc[idx]["image_id"])
        )
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        image = self.transform(image=image)["image"]

        return image

In [72]:
transforms = A.Compose([A.Resize(height=128, width=128), A.Normalize(), ToTensorV2()])

data_set = ImageData(
    data=df,
    directory=path,
    transform=transforms,
    actual_classes=actual_classes
)

data_loader = DataLoader(data_set, batch_size=32, shuffle=False, num_workers=2)

In [73]:
model = timm.create_model(model_name="resnet18", pretrained=True)
model.fc = nn.Linear(512, 4)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [74]:
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()

    return hook
model.global_pool.register_forward_hook(get_features("feats"))

In [75]:
PREDS = []
FEATS = []

features = {}

for idx, inputs in enumerate(data_loader):

    inputs = inputs.to(device)

    preds = model(inputs)

    PREDS.append(preds.detach().cpu().numpy())
    FEATS.append(features["feats"].cpu().numpy())


In [76]:
features_df = pd.DataFrame(columns=list(range(512)))

In [77]:
for i in range(len(FEATS)):
  for j in range(len(FEATS[i])):
    features_df.loc[len(features_df)] = list(FEATS[i][j])

In [78]:
features_df['label'] = df['label']

# 2. Dimension Reduction

In [79]:
pca = PCA(n_components=2)

transformed_data = pca.fit_transform(features_df.drop('label', axis=1))

In [80]:
transformed_data

array([[-2.032571  ,  4.191373  ],
       [-1.6878549 ,  1.7816083 ],
       [-0.74498135, -1.8623611 ],
       ...,
       [-0.6238642 ,  1.2101009 ],
       [-0.00475764,  1.4298079 ],
       [-2.19      , -0.01429355]], dtype=float32)

# 3. Clustering Algorithm

In [81]:
# K-means clustering with 'random'
kmeans_random = KMeans(n_clusters=4, init='random', random_state=20)
kmeans_random.fit(transformed_data)
kmeans_random_labels = kmeans_random.labels_

# K-means clustering with 'k-means++'
kmeans_kmeans_pp = KMeans(n_clusters=4, init='k-means++', random_state=20)
kmeans_kmeans_pp.fit(transformed_data)
kmeans_kmeans_pp_labels = kmeans_kmeans_pp.labels_

# Bisecting K-means clustering with 'random'
bisecting_kmeans_random = BisectingKMeans(n_clusters=4, init='random', random_state=20)
bisecting_kmeans_random.fit(transformed_data)
bisecting_kmeans_random_labels = bisecting_kmeans_random.labels_

# Spectral clustering with default parameters
spectral_clustering = SpectralClustering(n_clusters=4, random_state=20)
spectral_clustering.fit(transformed_data)
spectral_clustering_labels = spectral_clustering.labels_

In [91]:
# DBSCAN
dbscan = DBSCAN(eps=0.4, min_samples=10)
dbscan.fit(transformed_data)
dbscan_labels = dbscan.labels_

# Agglomerative clustering with different linkage methods
agglomerative_single = AgglomerativeClustering(n_clusters=4, linkage='single')
agglomerative_single.fit(transformed_data)
agglomerative_single_labels = agglomerative_single.labels_

agglomerative_complete = AgglomerativeClustering(n_clusters=4, linkage='complete')
agglomerative_complete.fit(transformed_data)
agglomerative_complete_labels = agglomerative_complete.labels_

agglomerative_average = AgglomerativeClustering(n_clusters=4, linkage='average')
agglomerative_average.fit(transformed_data)
agglomerative_average_labels = agglomerative_average.labels_

agglomerative_ward = AgglomerativeClustering(n_clusters=4, linkage='ward')
agglomerative_ward.fit(transformed_data)
agglomerative_ward_labels = agglomerative_ward.labels_

In [93]:
np.unique(dbscan_labels)

# eps is equal to 0.4 and min_samples parameter is 10 to get 4 clusters

array([-1,  0,  1,  2])

# 4. Clustering Evaluations

In [94]:
orginal_labels = features_df['label']

In [95]:

# Calculate Fowlkes-Mallows index
fowlkes_mallows_scores = {
    'K-means (Random)': fowlkes_mallows_score(orginal_labels, kmeans_random_labels),
    'K-means (k-means++)': fowlkes_mallows_score(orginal_labels, kmeans_kmeans_pp_labels),
    'Bisecting K-means': fowlkes_mallows_score(orginal_labels, bisecting_kmeans_random_labels),
    'Spectral Clustering': fowlkes_mallows_score(orginal_labels, spectral_clustering_labels),
    'DBSCAN': fowlkes_mallows_score(orginal_labels, dbscan_labels),
    'Agglomerative (Single link- MIN)': fowlkes_mallows_score(orginal_labels, agglomerative_single_labels),
    'Agglomerative (Complete link- MAX)': fowlkes_mallows_score(orginal_labels, agglomerative_complete_labels),
    'Agglomerative (Group Average)': fowlkes_mallows_score(orginal_labels, agglomerative_average_labels),
    'Agglomerative (Ward)': fowlkes_mallows_score(orginal_labels, agglomerative_ward_labels)
}


In [96]:
# Calculate Silhouette Coefficient
silhouette_scores = {
    'K-means (Random)': silhouette_score(transformed_data, kmeans_random_labels),
    'K-means (k-means++)': silhouette_score(transformed_data, kmeans_kmeans_pp_labels),
    'Bisecting K-means': silhouette_score(transformed_data, bisecting_kmeans_random_labels),
    'Spectral Clustering': silhouette_score(transformed_data, spectral_clustering_labels),
    'DBSCAN': silhouette_score(transformed_data, dbscan_labels),
    'Agglomerative (Single link-MIN)': silhouette_score(transformed_data, agglomerative_single_labels),
    'Agglomerative (Complete link-MAX)': silhouette_score(transformed_data, agglomerative_complete_labels),
    'Agglomerative (Group Average)': silhouette_score(transformed_data, agglomerative_average_labels),
    'Agglomerative (Ward)': silhouette_score(transformed_data, agglomerative_ward_labels)
}

In [97]:
# Rank methods based on Fowlkes-Mallows index
ranked_methods_fm = sorted(fowlkes_mallows_scores.items(), key=lambda x: x[1], reverse=True)
print("Ranking based on Fowlkes-Mallows index:")
for method, score in ranked_methods_fm:
    print(f"{method}: {score}")

Ranking based on Fowlkes-Mallows index:
Agglomerative (Single link- MIN): 0.4982544375031002
Spectral Clustering: 0.3855319029938198
DBSCAN: 0.3755390768771918
Agglomerative (Complete link- MAX): 0.36929803127961575
Agglomerative (Group Average): 0.35085569553892687
K-means (k-means++): 0.26458420928708204
K-means (Random): 0.2598071716208636
Bisecting K-means: 0.2589423885177832
Agglomerative (Ward): 0.2571493780301932


In [98]:
# Rank methods based on Silhouette Coefficient
ranked_methods_silhouette = sorted(silhouette_scores.items(), key=lambda x: x[1], reverse=True)
print("\nRanking based on Silhouette Coefficient:")
for method, score in ranked_methods_silhouette:
    print(f"{method}: {score}")


Ranking based on Silhouette Coefficient:
Agglomerative (Single link-MIN): 0.39202287793159485
Spectral Clustering: 0.39007893204689026
K-means (k-means++): 0.36184126138687134
Agglomerative (Group Average): 0.3497146666049957
K-means (Random): 0.33136042952537537
Bisecting K-means: 0.3300669193267822
Agglomerative (Ward): 0.3030272424221039
Agglomerative (Complete link-MAX): 0.2649152874946594
DBSCAN: -0.045270971953868866
